### Predicting past performance

TOC:

* [Daily eur output since january 2021](#daily_output) Calculates daily income for the specified mining config
* [FPPS vs. PPLNS risk-reward tradeoff](#risk_reward) Reward distributions under different mining contracts

This combines 
* historical blockchain data 
* historical hashrate data
* historical elecricity price data in Spain
* a miner configurating specified in terms of hashrate and kWh per day


to calculate the performance of a mining configuration over the past year or so under fpps and pplns reward schemes.

Blockchain data from https://bitcointalk.org/index.php?topic=5246271.msg54373224#msg54373224


In [1]:
import gzip
import csv
import pandas as pd
from matplotlib import pyplot as plt
import datetime as dt
import re
import numpy as np

import bokeh
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
def load_blockchain():
    df = pd.read_csv('blockchain_data.csv')
    df['day'] = pd.to_datetime(df['day'])
    return df

def load_hashrate_th_s():
    """
    Unit: terahashes/second
    """
    df = pd.read_csv('hash-rate.csv')
    df['day'] = [dt.datetime.strptime(stamp.split()[0], '%Y-%m-%d') for stamp in df['Timestamp']]
    df = df[['day', 'hash-rate']]
    df = df.rename(columns={'hash-rate': 'terahashes-second'})
    return df

def load_btc_eur():
    df= pd.read_csv('BTC-EUR.csv')
    df['day'] = [dt.datetime.strptime(d, '%Y-%m-%d') for d in df['Date']]
    df = df[['day','Open', 'Close']]
    df = df.rename(columns={'Open': 'btc_eur_open', 'Close': 'btc_eur_close'})
    return df.dropna()

def load_electricity():
    meses = 'enero febrero marzo abril mayo junio julio agosto septiembre octubre'.split()
    months = {mes: i+1 for i, mes in enumerate(meses)}
    
    def parse_periodo(text):
        m = re.match("Día ([0-9]+) de (.+)", text)
        day = int(m[1])
        mes = m[2]
        month = months[mes]
        
        return dt.datetime(2021, month, day)
    
    def spanish_to_sane(num):
        return float('.'.join(num.split(',')))
        
    df = pd.read_csv('evolucion_diaria_del_precio_del_mwh_en_españa.csv', delimiter=';')
    df['day'] = [parse_periodo(p) for p in df['Periodo']]
    df['eur/mWh'] = [spanish_to_sane(num) for num in df['Euros']]
    df = df[['day', 'eur/mWh']]
    return df
    

In [3]:
block_df = load_blockchain()
hashrate_df = load_hashrate_th_s()
btc_eur_df = load_btc_eur()
eur_mWh_df = load_electricity()

In [4]:
with_rate_df = block_df.merge(hashrate_df, how='inner', on='day')
df = with_rate_df.merge(btc_eur_df, how='inner', on='day')
df = df.merge(eur_mWh_df, how='inner', on='day')

In [5]:
df

,id,time,difficulty,fee_total,generation,guessed_miner,day,terahashes-second,btc_eur_open,btc_eur_close,eur/mWh
0,664062,2021-01-02 00:09:23,1.859959e+13,48172334,625000000,F2Pool,2021-01-02,1.460853e+08,24167.451172,26474.892578,48.72
1,664063,2021-01-02 00:26:26,1.859959e+13,40088251,625000000,F2Pool,2021-01-02,1.460853e+08,24167.451172,26474.892578,48.72
2,664064,2021-01-02 00:34:05,1.859959e+13,19624901,625000000,Huobi,2021-01-02,1.460853e+08,24167.451172,26474.892578,48.72
3,664065,2021-01-02 00:46:22,1.859959e+13,31568122,625000000,BTC.com,2021-01-02,1.460853e+08,24167.451172,26474.892578,48.72
4,664066,2021-01-02 00:52:11,1.859959e+13,13895953,625000000,OKEX,2021-01-02,1.460853e+08,24167.451172,26474.892578,48.72
...,...,...,...,...,...,...,...,...,...,...,...
36946,701322,2021-09-19 22:54:32,1.841516e+13,4075062,625000000,AntPool,2021-09-19,1.464676e+08,41164.308594,40292.125000,146.57
36947,701323,2021-09-19 23:02:18,1.841516e+13,7918142,625000000,Unknown,2021-09-19,1.464676e+08,41164.308594,40292.125000,146.57
36948,701324,2021-09-19 23:10:28,1.841516e+13,6512559,625000000,SlushPool,2021-09-19,1.464676e+08,41164.308594,40292.125000,146.57
36949,701325,2021-09-19 23:34:32,1.841516e+13,12496513,625000000,AntPool,2021-09-19,1.464676e+08,41164.308594,40292.125000,146.57


In [6]:
def global_fpps(terahashes_sec, df):
    return (terahashes_sec * (df['fee_total'] + df['generation'])) / df['terahashes-second']

def sample_pplns(global_pool_share, terahashes_sec, df):
    my_share = terahashes_sec / (global_pool_share * df['terahashes-second'])
    successes = np.random.binomial(1, global_pool_share, df['id'].shape)
    satoshis = my_share * (df['fee_total'] + df['generation']) * successes
    btcs = satoshis / 100000000
    eurs = btcs * df['btc_eur_close']
    return eurs
    
    

In [7]:
TITLE="14x Antminer S19 JPRO 104 TH/s"
TERAHASHES_SEC = 100 * 14
FPPS_TAX = 0.025
kWh_day = 74.88 * 14

df['global_fpps'] = (1 - FPPS_TAX) * global_fpps(TERAHASHES_SEC, df)
df['global_fpps_btc'] = df['global_fpps'] / 100000000
df['fpps_eur'] = df['global_fpps_btc'] * df['btc_eur_close']

agg_df = df[['day', 'global_fpps', 'global_fpps_btc', 'fpps_eur']].groupby('day').sum()
agg_df = agg_df.reset_index()


agg_df = agg_df.merge(eur_mWh_df, how='inner', on='day')
agg_df['eur_electricity'] = (kWh_day * agg_df['eur/mWh']) / 1000
agg_df['eur_profit'] = agg_df['fpps_eur'] - agg_df['eur_electricity']
agg_df

,day,global_fpps,global_fpps_btc,fpps_eur,eur/mWh,eur_electricity,eur_profit
0,2021-01-02,9.562958e+05,0.009563,253.178280,48.72,51.074150,202.104129
1,2021-01-03,9.164503e+05,0.009165,245.248704,46.93,49.197658,196.051046
2,2021-01-04,1.118959e+06,0.011190,291.981103,59.85,62.741952,229.239151
3,2021-01-05,9.130017e+05,0.009130,252.348931,67.55,70.814016,181.534915
4,2021-01-06,1.085074e+06,0.010851,323.838932,70.60,74.011392,249.827540
...,...,...,...,...,...,...,...
254,2021-09-14,9.451118e+05,0.009451,377.102391,153.43,160.843738,216.258654
255,2021-09-15,9.498598e+05,0.009499,387.183303,172.78,181.128730,206.054573
256,2021-09-16,9.503513e+05,0.009504,385.987842,188.18,197.272858,188.714984
257,2021-09-17,9.471429e+05,0.009471,381.817714,166.29,174.325133,207.492581


<a id='daily_output'></a>
### Daily eur output since january 2021

In [8]:
source = ColumnDataSource(agg_df)
p = figure(title = TITLE)
p.yaxis.axis_label = 'euros'
p.xaxis.axis_label = 'day'

p.scatter("day", "eur_profit", source=source, legend_label="profit")
p.line("day", "eur_electricity", source=source, line_color="red", legend_label="electricity")
p.line("day", "fpps_eur", source=source, line_color="green", legend_label="fpps_euros")
p.xaxis.formatter = bokeh.models.DatetimeTickFormatter(hourmin = ['%H:%M']) 
show(p)


In [9]:
pd.to_datetime(agg_df['day'])

0     2021-01-02
1     2021-01-03
2     2021-01-04
3     2021-01-05
4     2021-01-06
         ...    
254   2021-09-14
255   2021-09-15
256   2021-09-16
257   2021-09-17
258   2021-09-19
Name: day, Length: 259, dtype: datetime64[ns]

In [ ]:
pplns_samples = []
pplns_samples_30 = []
for i in range(50000):
    pplns_samples.append(sample_pplns(0.15, TERAHASHES_SEC, df).sum())
    pplns_samples_30.append(sample_pplns(0.30, TERAHASHES_SEC, df).sum())

In [ ]:
agg_df['fpps_eur'].sum()

In [ ]:
total_electricity_cost = agg_df['eur_electricity'].sum()
binance_profit = agg_df['eur_profit'].sum()
pplns_profits = np.array(pplns_samples) - agg_df['eur_electricity'].sum()
pplns_profits_30 = np.array(pplns_samples_30) - agg_df['eur_electricity'].sum()

In [ ]:
binance_profit

<a id='risk_reward'></a>
### FPPS vs. PPLNS risk-reward tradeoff

This compares reward distributions of:

* a binance pool-like 2.5 % fpps reward 
* an antpool-like 0 % pplns reward
* a hypothetical 0 % pplns reward in a pool with 30 % of global hash capacity



In [ ]:

hist, edges = np.histogram(pplns_profits, density=True, bins=50)
hist30, edges30 = np.histogram(pplns_profits_30, density=True, bins=50)
p = figure(title = "total profit distribution over trial period")
p.yaxis.axis_label = 'density'
p.xaxis.axis_label = 'euros'

p.quad(top=hist, 
       bottom=0, 
       left=edges[:-1], 
       right=edges[1:], 
       line_color="white", 
       legend_label='0% pplns in antpool (15% total hash capacity)', 
       fill_color = 'green', 
       fill_alpha=0.4)

p.quad(top=hist30, 
       bottom=0, 
       left=edges30[:-1], 
       right=edges30[1:], 
       line_color="white", 
       legend_label='0% pplns 30% total hash capacity', 
       fill_color = 'blue', 
       fill_alpha=0.4)


span = 0.5 * (edges[1] - edges[0])
fpps = p.quad(
    top=np.max(hist), 
    bottom=0, 
    left=[binance_profit -span], 
    right=[binance_profit+span], 
    line_color="white", 
    legend_label='2.5% fpps binance pool', 
    fill_color='red')

show(p)

In [ ]:
P_pplns_15_lt_fpps = 100 * (pplns_profits < binance_profit).sum() / pplns_profits.shape[0]
print("P(pplns15 < fpps) = {:.2f}%".format(P_pplns_15_lt_fpps))

In [ ]:
P_pplns_30_lt_fpps = 100 * (pplns_profits_30 < binance_profit).sum() / pplns_profits_30.shape[0]
print("P(pplns30 < fpps) = {:.2f}%".format(P_pplns_30_lt_fpps))

In [ ]:
df['fee_total'].mean() / 100000000